# Direct calculation of cfht_minimal fit

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import scipy.optimize

import matplotlib.pyplot as plt

The notation here follows that of [dmtn-036](http://dmtn-036.lsst.io).

## Flux calculation

To see the comparison numbers, `setup jointcal` and run this in `tests/`:

```
pytest -sv test_jointcal_cfht_minimal.py::JointcalTestCFHTMinimal::test_jointcalTask_2_visits_photometry
```

In [2]:
def residualMeas(scale, flux, fittedFlux):
    return scale*flux - fittedFlux

def invSigma(scale, err):
    return 1.0/(scale*err)


def measChi2(scale, flux, fluxErr, fittedFlux):
    return (residualMeas(scale, flux, fittedFlux) * invSigma(scale, fluxErr))**2

def computeChi2Meas(scale, flux1, fluxErr1, flux2, fluxErr2, fittedFlux, verbose=False):
    if verbose:
        print(measChi2(scale[0], flux1[0], fluxErr1[0], fittedFlux[0]))
        print(measChi2(scale[1], flux2[0], fluxErr2[0], fittedFlux[0]))
        print(measChi2(scale[1], flux2[1], fluxErr2[1], fittedFlux[1]))

    return measChi2(scale[0], flux1[0], fluxErr1[0], fittedFlux[0]) + \
            measChi2(scale[1], flux2[0], fluxErr2[0], fittedFlux[0]) + \
            measChi2(scale[1], flux2[1], fluxErr2[1], fittedFlux[1])


def refChi2(fittedFlux, refFlux, refFluxErr):
    return ((fittedFlux - refFlux)/refFluxErr)**2

def computeChi2Ref(fittedFlux, refFlux, refFluxErr, verbose=False):
    if verbose:
        print(refChi2(fittedFlux[0], refFlux[0], refFluxErr[0]))
        print(refChi2(fittedFlux[1], refFlux[1], refFluxErr[1]))

    return refChi2(fittedFlux[0], refFlux[0], refFluxErr[0]) + refChi2(fittedFlux[1], refFlux[1], refFluxErr[1])


def doChi2(scale, flux1, fluxErr1, flux2, fluxErr2, fitted, refFlux, refFluxErr, verbose=False):
    chi2Meas = computeChi2Meas(scale, flux1, fluxErr1, flux2, fluxErr2, fitted, verbose=verbose)
    chi2Ref = computeChi2Ref(fitted, refFlux, refFluxErr, verbose=verbose)
    print("chi2:", chi2Meas, " + ", chi2Ref, " = ", chi2Meas+chi2Ref)

These values come from the `-meas` and `-ref` files output by `saveChi2Contributions()` for the `cfht_minimal` test dataset, converted to maggies using the `processCcd` calibrations, so that the fit here is done as a ~1 scale factor. One CCD has one star, the other has two, making for two measurements of one star (flux~=3e-9) and one measurement of the other star (flux~=5.9e-9). Both stars have a reference catalog match.

The output of this calculation should identically match the `Initialized: chi2/ndof : 9.98915/5=1.99783` line in the log from just after the SimpleFluxModel was instantiated.

In [3]:
useMaggies = True

# ccd1
if useMaggies:
    flux1 = np.array([3.23755827e-09,])
    fluxErr1 = np.array([9.44805263e-11,])
else:
    flux1 = np.array([21234.2649])
    fluxErr1 = np.array([619.672096])

# ccd2
if useMaggies:
    flux2 = np.array([2.99096178e-09, 5.94452146e-09])
    fluxErr2 = np.array([8.09248684e-11, 8.34393948e-11,])
else:
    flux2 = np.array([20411.6636, 40568.0784])
    fluxErr2 = np.array([552.267567, 569.427822])

# stars 1 and 2
fittedFlux = np.array([3.11426003e-09, 5.94452146e-09])

# stars 1 and 2
refFlux = np.array([2.6954972e-09, 6.01832176e-09])
refFluxErr = np.array([1.73692254e-10, 1.89275075e-10])

# ccds 1 and 2
if useMaggies:
    scaleFlux = [1, 1]
else:
    scaleFlux = [1.524685827e-13,  1.465319951e-13]

doChi2(scaleFlux, flux1, fluxErr1, flux2, fluxErr2, fittedFlux, refFlux, refFluxErr, verbose=True)

1.70305685641
2.32139915248
0.0
5.81266575038
0.15203003679
chi2: 4.02445600888  +  5.96469578717  =  9.98915179606


### What does scipy say?

If `scipy.minimize` can solve the system, the final chi2 should match the jointcal result: `chi2/ndof : 2.3369/1=2.3369`

In [4]:
def computeChi2(scale):
    """scale 0,1 is the scale factor, 2,3 is the fittedFlux."""
    chi2Meas = computeChi2Meas(scale[:2], flux1, fluxErr1, flux2, fluxErr2, scale[2:], verbose=False)
    chi2Ref = computeChi2Ref(scale[2:], refFlux, refFluxErr, verbose=False)
    return chi2Meas + chi2Ref

scale0 = np.hstack((scaleFlux, fittedFlux))
result = scipy.optimize.minimize(computeChi2, scale0, method="Nelder-Mead")
print(result)
doChi2(result.x[:2], flux1, fluxErr1, flux2, fluxErr2, result.x[2:], refFlux, refFluxErr)

 final_simplex: (array([[  8.98343519e-01,   9.87452406e-01,   2.90832290e-09,
          5.89342202e-09],
       [  8.98315986e-01,   9.87400861e-01,   2.90839364e-09,
          5.89327092e-09],
       [  8.98353168e-01,   9.87479340e-01,   2.90859347e-09,
          5.89377338e-09],
       [  8.98415360e-01,   9.87450279e-01,   2.90858946e-09,
          5.89346997e-09],
       [  8.98426727e-01,   9.87434674e-01,   2.90846695e-09,
          5.89360071e-09]]), array([ 2.33676853,  2.33676952,  2.33677034,  2.33677376,  2.33677662]))
           fun: 2.3367685264535796
       message: 'Optimization terminated successfully.'
          nfev: 158
           nit: 93
        status: 0
       success: True
             x: array([  8.98343519e-01,   9.87452406e-01,   2.90832290e-09,
         5.89342202e-09])
chi2: 0.399952333245  +  1.93681619321  =  2.33676852645


### Simulating the jointcal computations, step by step

The output of this calculation should match the chi2 from after the first fitting step, just after "assignIndices: now fitting Model" (`chi2/ndof : 7.76225/3=2.58742`). Note that the derivative of this transform (`scale*flux`) w.r.t. the scale factor is just the flux.

In [5]:
def computeJMeas(scale, flux, fluxErr):
    return invSigma(scale, fluxErr) * flux

def computeGradMeas(scale, flux, fluxErr, fittedFlux):
    return flux * invSigma(scale, fluxErr)**2 * residualMeas(scale, flux, fittedFlux)

def computeJandGradMeas(scale, flux1, fluxErr1, flux2, fluxErr2, fitted):
    J = np.zeros((2,3))  # elements of the Jacobian are dD(f)/d(f0) * 1/sigma
    grad = np.zeros((2))  # elements of the gradient are -d chi2(f)/d(f0) = -dD(f)/d(f0) * (1/sigma)^2 * D(f)

    # contributions from ccd1
    i = 0
    J[i, 0] = computeJMeas(scale[i], flux1[0], fluxErr1[0])
    grad[i] += computeGradMeas(scale[i], flux1[0], fluxErr1[0], fitted[0])

    # contributions from ccd2
    i = 1
    J[i, 1] = computeJMeas(scale[i], flux2[0], fluxErr2[0])
    grad[i] += computeGradMeas(scale[i], flux2[0], fluxErr2[0], fitted[0])
    J[i, 2] = computeJMeas(scale[i], flux2[1], fluxErr2[1])
    grad[i] += computeGradMeas(scale[i], flux2[1], fluxErr2[1], fitted[1])

    return J, grad

def solve(grad, J):
    print("grad:", grad)
    print("Jacobian:\n", J)
    Hessian = np.dot(J,J.T)
    print("Hessian = J^T*J:\n", Hessian)
    delta = np.linalg.solve(Hessian,-grad)
    print("delta:", delta)
    return delta

In [6]:
if useMaggies:
    scaleFlux = [1,  1]
else:
    scaleFlux = [1.524685827e-13,  1.465319951e-13]

def doOne(scale, flux1, fluxErr1, flux2, fluxErr2, fittedFlux, refFlux, refFluxErr):
    J, grad = computeJandGradMeas(scale, flux1, fluxErr1, flux2, fluxErr2, fittedFlux)
    newScale = scale + solve(grad, J)
    print("new scale:", newScale)
    doChi2(newScale, flux1, fluxErr1, flux2, fluxErr2, fittedFlux, refFlux, refFluxErr)
    return newScale

print("Take one step:")
scaleFlux = doOne(scaleFlux, flux1, fluxErr1, flux2, fluxErr2, fittedFlux, refFlux, refFluxErr)
print()
print("The second step should not move within machine precision:")
scaleFlux = doOne(scaleFlux, flux1, fluxErr1, flux2, fluxErr2, fittedFlux, refFlux, refFluxErr)

Take one step:
grad: [ 44.71877141 -56.31236568]
Jacobian:
 [[ 34.26693729   0.           0.        ]
 [  0.          36.9597361   71.24358313]]
Hessian = J^T*J:
 [[ 1174.22299135     0.        ]
 [    0.          6441.67023022]]
delta: [-0.03808371  0.00874189]
new scale: [ 0.96191629  1.00874189]
chi2: 1.79755724436  +  5.96469578717  =  7.76225303154

The second step should not move within machine precision:
grad: [  0.00000000e+00  -7.10542736e-14]
Jacobian:
 [[ 35.62361689   0.           0.        ]
 [  0.          36.6394382   70.62617694]]
Hessian = J^T*J:
 [[ 1269.04208059     0.        ]
 [    0.          6330.50530065]]
delta: [ -0.00000000e+00   1.12241077e-17]
new scale: [ 0.96191629  1.00874189]
chi2: 1.79755724436  +  5.96469578717  =  7.76225303154


Now take a step along the refstar direction. The output of this calculation should match the chi2 from after the second fitting step, just after "assignIndices: now fitting Flux" (`chi2/ndof : 4.57832/3=1.52611`).

In [7]:
def computeJRef(refFluxErr):
    return 1.0/refFluxErr

def computeGradRef(refFlux, refFluxErr, fittedFlux):
    return (fittedFlux - refFlux) * (1.0/refFluxErr)**2

def computeJandGradRef(scale, flux1, fluxErr1, flux2, fluxErr2, refFlux, refFluxErr, fittedFlux):
    J = np.zeros((2,5))  # 2 stars, 5 contributions (1 per fittedStar (2), 1 per measuredStar (3))
    grad = np.zeros((2))
        
    # Add in the contributions from (measuredStar-fittedStar)
    star = 0
    ccd = 0
    grad[star] += -1.0 * (invSigma(scale[ccd], fluxErr1[star]))**2 * residualMeas(scale[ccd], flux1[star], fittedFlux[star])
    J[star,0] += -1.0 * invSigma(scale[ccd], fluxErr1[star])

    ccd = 1
    grad[star] += -1.0 * (invSigma(scale[ccd], fluxErr2[star]))**2 * residualMeas(scale[ccd], flux2[star], fittedFlux[star])
    J[star,1] += -1.0 * invSigma(scale[ccd], fluxErr2[star])
    star = 1
    grad[star] += -1.0 * (invSigma(scale[ccd], fluxErr2[star]))**2 * residualMeas(scale[ccd], flux2[star], fittedFlux[star])
    J[star,2] += -1.0 * invSigma(scale[ccd], fluxErr2[star])

    # fittedStar components go at the end of the matrix (starting with column 3)
    for i, (fitted, ref, refErr) in enumerate(zip(fittedFlux, refFlux, refFluxErr)):
        J[i, i+3] = computeJRef(refFluxErr[i])
        grad[i] += computeGradRef(refFlux[i], refFluxErr[i], fittedFlux[i])

    return J, grad

In [8]:
def doOne(scale, flux1, fluxErr1, flux2, fluxErr2, fittedFlux, refFlux, refFluxErr):
    J, grad = computeJandGradRef(scale, flux1, fluxErr1, flux2, fluxErr2, refFlux, refFluxErr, fittedFlux)
    print()
    fittedFlux += solve(grad, J)
    print("new fitted fluxes:", fittedFlux)
    doChi2(scale, flux1, fluxErr1, flux2, fluxErr2, fittedFlux, refFlux, refFluxErr, verbose=True)
    return refFlux

print("Take one step:")
refFlux = doOne(scaleFlux, flux1, fluxErr1, flux2, fluxErr2, fittedFlux, refFlux, refFluxErr)
print()
print("The second step should not move within machine precision:")
refFlux = doOne(scaleFlux, flux1, fluxErr1, flux2, fluxErr2, fittedFlux, refFlux, refFluxErr)

Take one step:

grad: [  2.84595031e+10  -9.39535133e+09]
Jacobian:
 [[ -1.10032357e+10  -1.22500523e+10   0.00000000e+00   5.75730913e+09
    0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00  -1.18808852e+10   0.00000000e+00
    5.28331583e+09]]
Hessian = J^T*J:
 [[  3.04281586e+20   0.00000000e+00]
 [  0.00000000e+00   1.69068859e+20]]
delta: [ -9.35301523e-11   5.55711524e-11]
new fitted fluxes: [  3.02072988e-09   6.00009261e-09]
1.05911743994
0.00196806278507
0.00183426219563
3.50612541926
0.00927568233672
chi2: 1.06291976492  +  3.51540110159  =  4.57832086652

The second step should not move within machine precision:

grad: [  1.33514404e-05   2.90274620e-05]
Jacobian:
 [[ -1.10032357e+10  -1.22500523e+10   0.00000000e+00   5.75730913e+09
    0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00  -1.18808852e+10   0.00000000e+00
    5.28331583e+09]]
Hessian = J^T*J:
 [[  3.04281586e+20   0.00000000e+00]
 [  0.00000000e+00   1.69068859e+20]]
delta: [ -4.38785685e-26  -1.71690175

## Magnitude calculation

Now the same calculation in magnitude space for the `SimpleMagnitudeModel`. To see the comparison numbers, `setup jointcal` and run this in `tests/`:

```
pytest -sv test_jointcal_cfht_minimal.py::JointcalTestCFHTMinimal::test_jointcalTask_2_visits_photometry_magnitude
```

In [9]:
def flux2Mag(flux):
    return -2.5 * np.log10(flux)

def fluxErr2MagErr(flux, fluxErr):
    return 2.5 / np.log(10.0) * fluxErr/flux

In [10]:
def residualMeas(scale, mag, fittedMag):
    return (mag + scale) - fittedMag

def invSigma(scale, err):
    return 1.0/err


def measChi2(scale, mag, magErr, fittedMag):
    return (residualMeas(scale, mag, fittedMag) * invSigma(scale, magErr))**2

def computeChi2Meas(scale, mag1, magErr1, mag2, magErr2, fittedMag, verbose=False):
    if verbose:
        print(measChi2(scale[0], mag1[0], magErr1[0], fittedMag[0]))
        print(measChi2(scale[1], mag2[0], magErr2[0], fittedMag[0]))
        print(measChi2(scale[1], mag2[1], magErr2[1], fittedMag[1]))

    return measChi2(scale[0], mag1[0], magErr1[0], fittedMag[0]) + \
            measChi2(scale[1], mag2[0], magErr2[0], fittedMag[0]) + \
            measChi2(scale[1], mag2[1], magErr2[1], fittedMag[1])


def refChi2(fittedMag, refMag, refMagErr):
    return ((fittedMag - refMag)/refMagErr)**2

def computeChi2Ref(fittedMag, refMag, refMagErr, verbose=False):
    if verbose:
        print(refChi2(fittedMag[0], refMag[0], refMagErr[0]))
        print(refChi2(fittedMag[1], refMag[1], refMagErr[1]))

    return refChi2(fittedMag[0], refMag[0], refMagErr[0]) + refChi2(fittedMag[1], refMag[1], refMagErr[1])


def doChi2(scale, mag1, magErr1, mag2, magErr2, fittedMag, refMag, refMagErr, verbose=False):
    chi2Meas = computeChi2Meas(scale, mag1, magErr1, mag2, magErr2, fittedMag, verbose=verbose)
    chi2Ref = computeChi2Ref(fittedMag, refMag, refMagErr, verbose=verbose)
    print("chi2:", chi2Meas, " + ", chi2Ref, " = ", chi2Meas+chi2Ref)

In [11]:
# these are all in original units, for consistency with jointcal's calculations.
flux1 = np.array([21234.2649])
fluxErr1 = np.array([619.672096])
flux2 = np.array([20411.6636, 40568.0784])
fluxErr2 = np.array([552.267567, 569.427822])
fittedFlux = np.array([3.11426003e-09, 5.94452146e-09])
refFlux = np.array([2.6954972e-09, 6.01832176e-09])
refFluxErr = np.array([1.73692254e-10, 1.89275075e-10])
scaleFlux = [1.524685827e-13,  1.465319951e-13]

# ccd1
mag1 = flux2Mag(flux1)
magErr1 = fluxErr2MagErr(flux1, fluxErr1)

# ccd2
mag2 = flux2Mag(flux2)
magErr2 = fluxErr2MagErr(flux2, fluxErr2)

# stars 1 and 2
fittedMag = flux2Mag(fittedFlux)

# stars 1 and 2
refMag = flux2Mag(refFlux)
refMagErr = fluxErr2MagErr(refFlux, refFluxErr)

# ccds 1 and 2
scaleMag = flux2Mag(scaleFlux)

doChi2(scaleMag, mag1, magErr1, mag2, magErr2, fittedMag, refMag, refMagErr, verbose=True)

1.77026114391
2.22918791849
4.07319307165e-15
5.02232445232
0.153915513173
chi2: 3.9994490624  +  5.17623996549  =  9.17568902789


## Magnitude-based jointcal minimization

In [12]:
def computeJMeas(scale, mag, magErr):
    """Note that the derivative w.r.t. scale is always 1."""
    return invSigma(scale, magErr) * 1

def computeGradMeas(scale, mag, magErr, fittedMag):
    """Note that the derivative w.r.t. scale is always 1."""
    return 1 * invSigma(scale, magErr)**2 * residualMeas(scale, mag, fittedMag)

def computeJandGradMeas(scale, mag1, magErr1, mag2, magErr2, fitted):
    J = np.zeros((2,3))  # elements of the Jacobian are dD(f)/d(f0) * 1/sigma
    grad = np.zeros((2))  # elements of the gradient are -d chi2(f)/d(f0) = -dD(f)/d(f0) * (1/sigma)^2 * D(f)

    # contributions from ccd1
    i = 0
    J[i, 0] = computeJMeas(scale[i], mag1[0], magErr1[0])
    grad[i] += computeGradMeas(scale[i], mag1[0], magErr1[0], fitted[0])

    # contributions from ccd2
    i = 1
    J[i, 1] = computeJMeas(scale[i], mag2[0], magErr2[0])
    grad[i] += computeGradMeas(scale[i], mag2[0], magErr2[0], fitted[0])
    J[i, 2] = computeJMeas(scale[i], mag2[1], magErr2[1])
    grad[i] += computeGradMeas(scale[i], mag2[1], magErr2[1], fitted[1])

    return J, grad

def solve(grad, J):
    print("grad:", grad)
    print("Jacobian:\n", J)
    Hessian = np.dot(J,J.T)
    print("Hessian = J^T*J:\n", Hessian)
    delta = np.linalg.solve(Hessian,-grad)
    print("delta:", delta)
    return delta

In [13]:
scaleMag = flux2Mag(scaleFlux)

def doOne(scale, mag1, magErr1, mag2, magErr2, fittedMag, refMag, refMagErr):
    J, grad = computeJandGradMeas(scale, mag1, magErr1, mag2, magErr2, fittedMag)
    newScale = scale + solve(grad, J)
    print("new newScale:", newScale)
    doChi2(newScale, mag1, magErr1, mag2, magErr2, fittedMag, refMag, refMagErr)
    return newScale

print("Initial scale:", scaleMag)
print("Take one step:")
scaleMag = doOne(scaleMag, mag1, magErr1, mag2, magErr2, fittedMag, refMag, refMagErr)
print()
print("The second step should not move within machine precision:")
scaleMag = doOne(scaleMag, mag1, magErr1, mag2, magErr2, fittedMag, refMag, refMagErr)

Initial scale: [ 32.04204909  32.08516884]
Take one step:
grad: [-41.99229776  50.82505387]
Jacobian:
 [[ 31.56101566   0.           0.        ]
 [  0.          34.04117507  65.61776504]]
Hessian = J^T*J:
 [[  996.09770968     0.        ]
 [    0.          5464.49268845]]
delta: [ 0.04215681 -0.00930096]
new newScale: [ 32.0842059   32.07586788]
chi2: 1.75646588409  +  5.17623996549  =  6.93270584958

The second step should not move within machine precision:
grad: [  0.00000000e+00  -1.01536557e-11]
Jacobian:
 [[ 31.56101566   0.           0.        ]
 [  0.          34.04117507  65.61776504]]
Hessian = J^T*J:
 [[  996.09770968     0.        ]
 [    0.          5464.49268845]]
delta: [ -0.00000000e+00   1.85811497e-15]
new newScale: [ 32.0842059   32.07586788]
chi2: 1.75646588409  +  5.17623996549  =  6.93270584958


In [14]:
def computeJRef(refMagErr):
    return 1.0/refMagErr

def computeGradRef(refMag, refMagErr, fittedMag):
    return (fittedMag - refMag) * (1.0/refMagErr)**2

def computeJandGradRef(scale, mag1, magErr1, mag2, magErr2, refMag, refMagErr, fittedMag):
    J = np.zeros((2,5))  # 2 stars, 5 contributions (1 per fittedStar (2), 1 per measuredStar (3))
    grad = np.zeros((2))
        
    # Add in the contributions from (measuredStar-fittedStar)
    star = 0
    ccd = 0
    grad[star] += -1.0 * (invSigma(scale[ccd], magErr1[star]))**2 * residualMeas(scale[ccd], mag1[star], fittedMag[star])
    J[star,0] += -1.0 * invSigma(scale[ccd], magErr1[star])

    ccd = 1
    grad[star] += -1.0 * (invSigma(scale[ccd], magErr2[star]))**2 * residualMeas(scale[ccd], mag2[star], fittedMag[star])
    J[star,1] += -1.0 * invSigma(scale[ccd], magErr2[star])
    star = 1
    grad[star] += -1.0 * (invSigma(scale[ccd], magErr2[star]))**2 * residualMeas(scale[ccd], mag2[star], fittedMag[star])
    J[star,2] += -1.0 * invSigma(scale[ccd], magErr2[star])

    # fittedStar components go at the end of the matrix (starting with column 3)
    for i, (fitted, ref, refErr) in enumerate(zip(fittedMag, refMag, refMagErr)):
        J[i, i+3] = computeJRef(refMagErr[i])
        grad[i] += computeGradRef(refMag[i], refMagErr[i], fittedMag[i])

    return J, grad

In [15]:
def doOne(scale, mag1, magErr1, mag2, magErr2, fittedMag, refMag, refMagErr):
    J, grad = computeJandGradRef(scale, mag1, magErr1, mag2, magErr2, refMag, refMagErr, fittedMag)
    print()
    fittedMag += solve(grad, J)
    print("new fitted mages:", fittedMag)
    doChi2(scale, mag1, magErr1, mag2, magErr2, fittedMag, refMag, refMagErr, verbose=True)
    return refMag

print("Take one step:")
refMag = doOne(scaleMag, mag1, magErr1, mag2, magErr2, fittedMag, refMag, refMagErr)
print()
print("The second step should not move within machine precision:")
refMag = doOne(scaleMag, mag1, magErr1, mag2, magErr2, fittedMag, refMag, refMagErr)

Take one step:

grad: [-72.07926527  51.53652501]
Jacobian:
 [[-31.56101566 -34.04117507   0.          14.29335282   0.        ]
 [  0.           0.         -65.61776504   0.          29.28583802]]
Hessian = J^T*J:
 [[ 2359.19924435     0.        ]
 [    0.          5163.35139715]]
delta: [ 0.03055243 -0.00998122]
new fitted mages: [ 21.29716525  20.55472654]
0.929808265988
0.0186022866309
0.00199241674352
3.25570664799
0.0100024810892
chi2: 0.950402969362  +  3.26570912908  =  4.21611209845

The second step should not move within machine precision:

grad: [  5.71986902e-13  -8.84625706e-13]
Jacobian:
 [[-31.56101566 -34.04117507   0.          14.29335282   0.        ]
 [  0.           0.         -65.61776504   0.          29.28583802]]
Hessian = J^T*J:
 [[ 2359.19924435     0.        ]
 [    0.          5163.35139715]]
delta: [ -2.42449595e-16   1.71327814e-16]
new fitted mages: [ 21.29716525  20.55472654]
0.929808265988
0.0186022866309
0.00199241674352
3.25570664799
0.0100024810892
c

### What does scipy say?

This should identically match the final chi2 found by `test_jointcal_cfht_minimal.py::JointcalTestCFHTMinimal::test_jointcalTask_2_visits_photometry_magnitude`.

In [16]:
def computeChi2(scale):
    """scale 0,1 is the scale factor, 2,3 is the fittedMag."""
    chi2Meas = computeChi2Meas(scale[:2], mag1, magErr1, mag2, magErr2, scale[2:], verbose=False)
    chi2Ref = computeChi2Ref(scale[2:], refMag, refMagErr, verbose=False)
    return chi2Meas + chi2Ref

scale0 = np.hstack((scaleMag, fittedMag))
result = scipy.optimize.minimize(computeChi2, scale0, method="Nelder-Mead")
print(result)
doChi2(result.x[:2], mag1, magErr1, mag2, magErr2, result.x[2:], refMag, refMagErr)

 final_simplex: (array([[ 32.15464362,  32.09644933,  21.33701302,  20.57188732],
       [ 32.15455603,  32.09649287,  21.33702155,  20.57192763],
       [ 32.15467851,  32.09648142,  21.33707356,  20.57192751],
       [ 32.15463053,  32.09648843,  21.33703465,  20.57195249],
       [ 32.15470554,  32.09650618,  21.33704753,  20.57192873]]), array([ 2.23008175,  2.23008416,  2.23008544,  2.23008549,  2.23008642]))
           fun: 2.2300817523099341
       message: 'Optimization terminated successfully.'
          nfev: 148
           nit: 80
        status: 0
       success: True
             x: array([ 32.15464362,  32.09644933,  21.33701302,  20.57188732])
chi2: 0.342249262093  +  1.88783249022  =  2.23008175231
